<h1 align="center">Space-Lunch</h1>
A journey through coding with the Engineering Process

![EDP](https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Fwww.sciencebuddies.org%2Fblog%2Fgraphics%2F2013-blog-scientific-method-engineering-design-charts-750px.png&f=1&nofb=1)

# Defining our Problem

## Abstract
Look up places to eat near launch sites

## Objective
Create a Python Jupyter Notebook application that will allow a user to select upcoming SpaceX launches and then display places to eat back to the user on a map.

## Requirements
1. Allow user to select between upcoming SpaceX launches.
2. Display places to eat near selected launch back to user on a map.

## Project Steps:
1. [x] Get the list of upcoming launches including date/time and location. Data source: Probably SpaceX API
2. [x] Extract the relevant data (launchpad location, time of launch).
3. [x] Display the information to the user.
4. [x] Allow user to select between launches. (This gives us the launch location)
5. _ Use launch location to get the restaurants. Data Source: Probably Google API
6. _ Parse the restaurant data
7. _ Display the restaurants on a map.

## Research Data:
Most of our research will be on the data sources we intend to use, so we can link to the documentation for that here:

1. Data source for upcoming launches: https://api.spacexdata.com/v4/launches/upcoming
    * Launchpad value comes back as a very ugly ID number (i.e.: 5e9e4501f509094ba4566f84)
2. Data source to convert launchpad IDs to text: 
3. Data source for places to eat nearby: https://developers.google.com/maps/documentation/places/web-service/search-nearby
    * Required Parameter: Location - The point around which to retrieve place information. This must be specified as latitude,longitude.
    * As this requires our location be in lat/lng format, we will need to convert our launch locations to lat/lng. Use this: https://developers.google.com/maps/documentation/geocoding/requests-geocoding?hl=en

# Prototyping
This section of the notebook is designated just for prototyping and testing out pieces of code. Much of our code is "test as you go", testing out each each step in the project as we program it, but this section set aside for all of our messy experimentation while we try to figure out what kind of approach we want to take to building each step.

In [1]:
# Import Dependencies
import requests
import pandas as pd
import gmaps
import matplotlib.pyplot as plt
import datetime
import time

In [2]:
resp = requests.get("https://api.spacexdata.com/v4/launches/upcoming").json()
resp

[{'fairings': {'reused': None,
   'recovery_attempt': None,
   'recovered': None,
   'ships': []},
  'links': {'patch': {'small': 'https://imgur.com/BrW201S.png',
    'large': 'https://imgur.com/573IfGk.png'},
   'reddit': {'campaign': 'https://www.reddit.com/r/spacex/comments/jhu37i/starlink_general_discussion_and_deployment_thread/',
    'launch': None,
    'media': None,
    'recovery': 'https://www.reddit.com/r/spacex/comments/k2ts1q/rspacex_fleet_updates_discussion_thread/'},
   'flickr': {'small': [], 'original': []},
   'presskit': None,
   'webcast': None,
   'youtube_id': None,
   'article': None,
   'wikipedia': 'https://en.wikipedia.org/wiki/Starlink'},
  'static_fire_date_utc': None,
  'static_fire_date_unix': None,
  'net': False,
  'window': None,
  'rocket': '5e9d0d95eda69973a809d1ec',
  'success': None,
  'failures': [],
  'details': None,
  'crew': [],
  'ships': [],
  'capsules': [],
  'payloads': ['61fc02e1e0dc5662b76489b4'],
  'launchpad': '5e9e4501f509094ba4566f84'

In [3]:
date = resp[0]['date_local']
date

'2022-01-31T19:00:00-05:00'

In [4]:
pad = resp[0]['launchpad']
pad

'5e9e4501f509094ba4566f84'

In [5]:
# That launchpad ID is pretty gross, let's convert it to something we can use!
resp = requests.get("https://api.spacexdata.com/v4/launchpads/" + pad).json()
resp

{'images': {'large': ['https://i.imgur.com/9oEMXwa.png']},
 'name': 'CCSFS SLC 40',
 'full_name': 'Cape Canaveral Space Force Station Space Launch Complex 40',
 'locality': 'Cape Canaveral',
 'region': 'Florida',
 'latitude': 28.5618571,
 'longitude': -80.577366,
 'launch_attempts': 80,
 'launch_successes': 78,
 'rockets': ['5e9d0d95eda69973a809d1ec'],
 'timezone': 'America/New_York',
 'launches': ['5eb87cddffd86e000604b32f',
  '5eb87cdeffd86e000604b330',
  '5eb87cdfffd86e000604b331',
  '5eb87ce0ffd86e000604b332',
  '5eb87ce1ffd86e000604b333',
  '5eb87ce2ffd86e000604b335',
  '5eb87ce3ffd86e000604b336',
  '5eb87ce4ffd86e000604b337',
  '5eb87ce4ffd86e000604b338',
  '5eb87ce5ffd86e000604b339',
  '5eb87ce6ffd86e000604b33a',
  '5eb87ce7ffd86e000604b33b',
  '5eb87ce8ffd86e000604b33c',
  '5eb87ceaffd86e000604b33d',
  '5eb87ceaffd86e000604b33e',
  '5eb87cecffd86e000604b33f',
  '5eb87cedffd86e000604b340',
  '5eb87ceeffd86e000604b341',
  '5eb87cefffd86e000604b342',
  '5eb87cf2ffd86e000604b344',


# Parsing SpaceX Data
Now that we are done prototyping, let's build a block of code to output just the date and location for each upcoming SpaceX launch.

In [6]:
# Get list of upcoming launches from SpaceX API
resp = requests.get("https://api.spacexdata.com/v4/launches/upcoming").json()

# Get date/time and location for each launch
launches = [] # Create a list to store launch information in

for launch in resp: # For each "launch" dictionary in the "resp" list of launches:
    
    date = launch['date_local'] # Retrieve the date for that launch
    pad_id = launch['launchpad']# Retrieve the pad ID for that launch
    
    # Convert the pad ID into a pad name using an additional API call
    resp2 = requests.get("https://api.spacexdata.com/v4/launchpads/" + pad_id).json()
    pad = resp2['full_name'] # Pull the full pad name out of the resp2 dictionary
    
    # Put the date and pad name for this launch in a dictionary and add that to the launches list
    launches.append({'date': date, "pad": pad})

# Convert the launches list of dictionaries to a DataFrame
df = pd.DataFrame(launches)

# Display the list of launches to the user
df

,date,pad
0,2022-01-31T19:00:00-05:00,Cape Canaveral Space Force Station Space Launc...
1,2022-02-13T16:00:00-08:00,Vandenberg Space Force Base Space Launch Compl...
2,2022-02-14T19:00:00-05:00,Cape Canaveral Space Force Station Space Launc...
3,2022-02-24T19:00:00-05:00,Cape Canaveral Space Force Station Space Launc...
4,2022-03-30T20:00:00-04:00,Kennedy Space Center Historic Launch Complex 39A
5,2022-03-31T17:00:00-07:00,Vandenberg Space Force Base Space Launch Compl...


# Get User Selection

In [19]:
# Get the user selection
selection = int(input("Select a launch to visit by the index number: "))
selection

Select a launch to visit by the index number: 1


1

In [20]:
# Based on user selection, get the appropriate pad location.
location = df['pad'][selection]

# Display that pad back to the user
location

'Vandenberg Space Force Base Space Launch Complex 4E'

# Pull and Parse Restaurant Data
After we have finished retrieving the SpaceX launch Data and asking the user what launch they wish to visit, we can move on to retrieving places to eat near that launch site.

In [21]:
# Import our gkey (Do not forget to make your own config.py)
from config import gkey

In [22]:
# Google geocoding API
# Now we can convert the pad name into lat/lng coordinates using the google geocoding API.
resp = requests.get(f"https://maps.googleapis.com/maps/api/geocode/json?address={location}&key={gkey}").json()
resp

{'results': [{'address_components': [{'long_name': 'Lompoc',
     'short_name': 'Lompoc',
     'types': ['locality', 'political']},
    {'long_name': 'Santa Barbara County',
     'short_name': 'Santa Barbara County',
     'types': ['administrative_area_level_2', 'political']},
    {'long_name': 'California',
     'short_name': 'CA',
     'types': ['administrative_area_level_1', 'political']},
    {'long_name': 'United States',
     'short_name': 'US',
     'types': ['country', 'political']},
    {'long_name': '93436', 'short_name': '93436', 'types': ['postal_code']}],
   'formatted_address': 'Lompoc, CA 93436, USA',
   'geometry': {'location': {'lat': 34.632022, 'lng': -120.6106597},
    'location_type': 'GEOMETRIC_CENTER',
    'viewport': {'northeast': {'lat': 34.6333709802915,
      'lng': -120.6093107197085},
     'southwest': {'lat': 34.6306730197085, 'lng': -120.6120086802915}}},
   'place_id': 'ChIJJ2a7H_Ej7IARo2agpDYPibI',
   'types': ['establishment', 'point_of_interest']},
  {

In [23]:
# Pulling our lat/lng coordinates out of response
lat = resp['results'][0]['geometry']['location']['lat']
lng = resp['results'][0]['geometry']['location']['lng']
print(lat, lng)

34.632022 -120.6106597


In [24]:
# google maps restaurant API call
url = f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={lat}%2C{lng}&radius=20000&type=restaurant&key={gkey}"

resp = requests.get(url).json()
food_df = pd.DataFrame(resp['results'])
list(food_df.columns).index('name') # A way to find the index numbers of a column key
food_df

,business_status,geometry,icon,icon_background_color,icon_mask_base_uri,name,opening_hours,photos,place_id,plus_code,rating,reference,scope,types,user_ratings_total,vicinity,price_level
0,OPERATIONAL,"{'location': {'lat': 34.6382585, 'lng': -120.4...",https://maps.gstatic.com/mapfiles/place_api/ic...,#909CE1,https://maps.gstatic.com/mapfiles/place_api/ic...,O'Cairns Inn & Suites,{'open_now': True},"[{'height': 540, 'html_attributions': ['<a hre...",ChIJk0Finpoe7IARp2qFAU3FdMM,"{'compound_code': 'JHQ3+87 Lompoc, CA, USA', '...",4.5,ChIJk0Finpoe7IARp2qFAU3FdMM,GOOGLE,"[lodging, restaurant, food, point_of_interest,...",350,"940 East Ocean Avenue, Lompoc",NaN
1,OPERATIONAL,"{'location': {'lat': 34.6563423, 'lng': -120.4...",https://maps.gstatic.com/mapfiles/place_api/ic...,#FF9E67,https://maps.gstatic.com/mapfiles/place_api/ic...,Subway,{'open_now': True},"[{'height': 1152, 'html_attributions': ['<a hr...",ChIJI_QIK7Ye7IARUucGOj4PDRk,"{'compound_code': 'MG4R+GM Lompoc, CA, USA', '...",3.8,ChIJI_QIK7Ye7IARUucGOj4PDRk,GOOGLE,"[meal_takeaway, restaurant, food, point_of_int...",100,"1137, 10 North H Street Suite C Space D, Lompoc",1.0
2,OPERATIONAL,"{'location': {'lat': 34.7006721, 'lng': -120.4...",https://maps.gstatic.com/mapfiles/place_api/ic...,#FF9E67,https://maps.gstatic.com/mapfiles/place_api/ic...,Subway,{'open_now': True},"[{'height': 3024, 'html_attributions': ['<a hr...",ChIJO1f6MAgZ7IARC5GrTn2xv1g,"{'compound_code': 'PG2J+7Q Lompoc, CA, USA', '...",3.8,ChIJO1f6MAgZ7IARC5GrTn2xv1g,GOOGLE,"[meal_takeaway, restaurant, food, point_of_int...",39,"3734 Constellation Road Unit A2, Lompoc",1.0
3,OPERATIONAL,"{'location': {'lat': 34.6386571, 'lng': -120.4...",https://maps.gstatic.com/mapfiles/place_api/ic...,#FF9E67,https://maps.gstatic.com/mapfiles/place_api/ic...,Subway,{'open_now': True},"[{'height': 2268, 'html_attributions': ['<a hr...",ChIJCxLlBpEe7IARJGORwUG4PxI,"{'compound_code': 'JGQX+FQ Lompoc, CA, USA', '...",4.1,ChIJCxLlBpEe7IARJGORwUG4PxI,GOOGLE,"[meal_takeaway, restaurant, food, point_of_int...",176,"616 East Ocean Avenue, Lompoc",1.0
4,OPERATIONAL,"{'location': {'lat': 34.6611728, 'lng': -120.4...",https://maps.gstatic.com/mapfiles/place_api/ic...,#FF9E67,https://maps.gstatic.com/mapfiles/place_api/ic...,Subway,{'open_now': True},"[{'height': 1836, 'html_attributions': ['<a hr...",ChIJ5UhOuMwe7IARUbvPZ_Scrm4,"{'compound_code': 'MG6M+FM Lompoc, CA, USA', '...",4.0,ChIJ5UhOuMwe7IARUbvPZ_Scrm4,GOOGLE,"[meal_takeaway, restaurant, food, point_of_int...",103,"721 West Central Avenue, Lompoc",1.0
5,OPERATIONAL,"{'location': {'lat': 34.6391723, 'lng': -120.4...",https://maps.gstatic.com/mapfiles/place_api/ic...,#FF9E67,https://maps.gstatic.com/mapfiles/place_api/ic...,Rice Bowl,{'open_now': True},"[{'height': 4032, 'html_attributions': ['<a hr...",ChIJJw-nspQe7IARCiN0FsXrqGs,"{'compound_code': 'JGQR+MF Lompoc, CA, USA', '...",3.8,ChIJJw-nspQe7IARCiN0FsXrqGs,GOOGLE,"[restaurant, food, point_of_interest, establis...",104,"117 West Ocean Avenue, Lompoc",NaN
6,OPERATIONAL,"{'location': {'lat': 34.65832, 'lng': -120.457...",https://maps.gstatic.com/mapfiles/place_api/ic...,#FF9E67,https://maps.gstatic.com/mapfiles/place_api/ic...,Little Caesars Pizza,{'open_now': True},"[{'height': 4032, 'html_attributions': ['<a hr...",ChIJI_QIK7Ye7IARBZMgvMqnUfc,"{'compound_code': 'MG5V+85 Lompoc, CA, USA', '...",4.0,ChIJI_QIK7Ye7IARBZMgvMqnUfc,GOOGLE,"[meal_delivery, meal_takeaway, restaurant, foo...",258,"1133 North H Street, Lompoc",1.0
7,OPERATIONAL,"{'location': {'lat': 34.6541497, 'lng': -120.4...",https://maps.gstatic.com/mapfiles/place_api/ic...,#FF9E67,https://maps.gstatic.com/mapfiles/place_api/ic...,El Toro Bronco,{'open_now': True},"[{'height': 336, 'html_attributions': ['<a hre...",ChIJaZrh9bce7IARFnLl-3DMXww,"{'compound_code': 'MG3V+M2 Lompoc, CA, USA', '...",4.3,ChIJaZrh9bce7IARFnLl-3DMXww,GOOGLE,"[restaurant, food, point_of_interest, establis...",556,"1030 North H Street, Lompoc",2.0
8,OPERATIONAL,"{'location': {'lat': 34.736234

In [25]:
restaurants_df = food_df.loc[:,['name','geometry']]
restaurants_df.set_index('name')

,geometry
name,
O'Cairns Inn & Suites,"{'location': {'lat': 34.6382585, 'lng': -120.4..."
Subway,"{'location': {'lat': 34.6563423, 'lng': -120.4..."
Subway,"{'location': {'lat': 34.7006721, 'lng': -120.4..."
Subway,"{'location': {'lat': 34.6386571, 'lng': -120.4..."
Subway,"{'location': {'lat': 34.6611728, 'lng': -120.4..."
Rice Bowl,"{'location': {'lat': 34.6391723, 'lng': -120.4..."
Little Caesars Pizza,"{'location': {'lat': 34.65832, 'lng': -120.457..."
El Toro Bronco,"{'location': {'lat': 34.6541497, 'lng': -120.4..."
Burger King,"{'location': {'lat': 34.736234, 'lng': -120.53..."


In [26]:
# Building our DataFrame using the columns keys
restaurants_df = food_df[['name','geometry']]
restaurants_df 

,name,geometry
0,O'Cairns Inn & Suites,"{'location': {'lat': 34.6382585, 'lng': -120.4..."
1,Subway,"{'location': {'lat': 34.6563423, 'lng': -120.4..."
2,Subway,"{'location': {'lat': 34.7006721, 'lng': -120.4..."
3,Subway,"{'location': {'lat': 34.6386571, 'lng': -120.4..."
4,Subway,"{'location': {'lat': 34.6611728, 'lng': -120.4..."
5,Rice Bowl,"{'location': {'lat': 34.6391723, 'lng': -120.4..."
6,Little Caesars Pizza,"{'location': {'lat': 34.65832, 'lng': -120.457..."
7,El Toro Bronco,"{'location': {'lat': 34.6541497, 'lng': -120.4..."
8,Burger King,"{'location': {'lat': 34.736234, 'lng': -120.53..."
9,Lompoc Tortilla Shop,"{'location': {'lat': 34.6404674, 'lng': -120.4..."


In [27]:
lats = []
lngs = []
# coords = []
for row in restaurants_df['geometry']:
    lats.append(row['location']['lat'])
    lngs.append(row['location']['lng'])
    
print(lats)
print()
print(lngs)

[34.6382585, 34.6563423, 34.7006721, 34.6386571, 34.6611728, 34.6391723, 34.65832, 34.6541497, 34.736234, 34.6404674, 34.6382913, 34.73498290000001, 34.73544199999999, 34.7358884, 34.7393279, 34.6611647, 34.66256389999999, 34.6614545, 34.65673879999999, 34.6387008]

[-120.446835, -120.4583468, -120.4680021, -120.4505288, -120.4658426, -120.4587791, -120.457, -120.4573997, -120.535885, -120.4526479, -120.4722076, -120.5382088, -120.537535, -120.5381925, -120.5370432, -120.464071, -120.4639292, -120.4626642, -120.4593714, -120.4563457]


In [28]:
restaurants_df['lats'] = lats
restaurants_df.loc[:, 'lngs'] = lngs
restaurants_df

C:\Users\farad\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,name,geometry,lats,lngs
0,O'Cairns Inn & Suites,"{'location': {'lat': 34.6382585, 'lng': -120.4...",34.638258,-120.446835
1,Subway,"{'location': {'lat': 34.6563423, 'lng': -120.4...",34.656342,-120.458347
2,Subway,"{'location': {'lat': 34.7006721, 'lng': -120.4...",34.700672,-120.468002
3,Subway,"{'location': {'lat': 34.6386571, 'lng': -120.4...",34.638657,-120.450529
4,Subway,"{'location': {'lat': 34.6611728, 'lng': -120.4...",34.661173,-120.465843
5,Rice Bowl,"{'location': {'lat': 34.6391723, 'lng': -120.4...",34.639172,-120.458779
6,Little Caesars Pizza,"{'location': {'lat': 34.65832, 'lng': -120.457...",34.658320,-120.457000
7,El Toro Bronco,"{'location': {'lat': 34.6541497, 'lng': -120.4...",34.654150,-120.457400
8,Burger King,"{'location': {'lat': 34.736234, 'lng': -120.53...",34.736234,-120.535885
9,Lompoc Tortilla Shop,"{'location': {'lat': 34.6404674, 'lng': -120.4...",34.640467,-120.452648


In [29]:
restaurants_df[["lats","lngs"]] # Double "[]" for keying with a list

,lats,lngs
0,34.638258,-120.446835
1,34.656342,-120.458347
2,34.700672,-120.468002
3,34.638657,-120.450529
4,34.661173,-120.465843
5,34.639172,-120.458779
6,34.658320,-120.457000
7,34.654150,-120.457400
8,34.736234,-120.535885
9,34.640467,-120.452648


In [30]:
fig = gmaps.figure()
markers = gmaps.marker_layer(restaurants_df[["lats","lngs"]])
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))